<a href="https://colab.research.google.com/github/Nulikontik/ai-fface/blob/master/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22roop_face_replacement_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title 1) Проверяем видеокарту/Check graphics card
!nvidia-smi

Fri Dec  8 06:58:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#@title 2) Подготовка
!git clone https://github.com/Obnovlator3000/roop
%cd /content/roop
!pip install onnxruntime-gpu && pip install -r requirements.txt
!wget https://huggingface.co/thebiglaskowski/inswapper_128.onnx/resolve/main/inswapper_128.onnx

Cloning into 'roop'...
remote: Enumerating objects: 1262, done.
remote: Total 1262 (delta 0), reused 0 (delta 0), pack-reused 1262
Receiving objects: 100% (1262/1262), 90.63 MiB | 34.40 MiB/s, done.
Resolving deltas: 100% (773/773), done.
/content/roop
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
Ignoring onnxruntime: markers 'sys_platform == "darwin" and platform_machine != "arm64"' don't match your environment
Ignoring onnxruntime-silicon: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
Ignoring tensorflow: markers 'sys_platform == "darwin"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 10.8 MB/s eta 0:00:00
     ━━━━━

--2023-12-08 07:03:26--  https://huggingface.co/thebiglaskowski/inswapper_128.onnx/resolve/main/inswapper_128.onnx
Resolving huggingface.co (huggingface.co)... 108.138.246.71, 108.138.246.67, 108.138.246.85, ...
Connecting to huggingface.co (huggingface.co)|108.138.246.71|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/69/55/6955e7aef1f96409a7afeb15ed35cbcf5439916338eb5622c66f01d89d29944d/e4a3f08c753cb72d04e10aa0f7dbe3deebbf39567d4ead6dce08e98aa49e16af?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27inswapper_128.onnx%3B+filename%3D%22inswapper_128.onnx%22%3B&Expires=1702278206&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMjI3ODIwNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82OS81NS82OTU1ZTdhZWYxZjk2NDA5YTdhZmViMTVlZDM1Y2JjZjU0Mzk5MTYzMzhlYjU2MjJjNjZmMDFkODlkMjk5NDRkL2U0YTNmMDhjNzUzY2I3MmQwNGUxMGFhMGY3ZGJlM2RlZWJiZjM5NTY3ZDRlYWQ2ZG

In [3]:
#@title 3) Загружаем фото с лицом/Upload photo with face
%cd /content/roop
!mkdir files
%cd /content/roop/files
from google.colab import files
uploaded = files.upload()
for i in uploaded.keys():
  face_raw = "/content/roop/files/" + i
  break
!mv "$face_raw" /content/roop/photo1702019172.jpeg

/content/roop
/content/roop/files


Saving photo1702019172.jpeg to photo1702019172.jpeg


In [4]:
#@title 4) Загружаем видео/Upload video
%cd /content/roop/files
from google.colab import files
uploaded = files.upload()
for i in uploaded.keys():
  video_raw = "/content/roop/files/" + i
  break
!mv "$video_raw" /content/roop/files/video.mp4

/content/roop/files


Saving Концовка фильма _Человек-Паук 2002_ (1080p).mp4 to Концовка фильма _Человек-Паук 2002_ (1080p).mp4


In [6]:
#@title 5) Заменяем лицо/Face swap
#@markdown Заменять все лица:
swap_all_faces  = True #@param{type:"boolean"}
if swap_all_faces:
  all_faces = "--all-faces"
else:
  all_faces = ""
%cd /content/roop/
!touch /content/output.mp4
c = f"python run.py -f /content/roop/photo1702019172.jpeg -t /content/roop/files/video.mp4 -o /content/output.mp4 --gpu-vendor nvidia --gpu-cores 3 {all_faces}"
get_ipython().system(c)
from google.colab import files
files.download('/content/output.mp4')

/content/roop
2023-12-08 07:11:43.776590: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 07:11:44.953609: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-08 07:11:47.847524: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-08 07:11:47.849815: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA nod

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>